Project is computer vision project to see if a person is wearing a mask or not with web cam.

Let us first import all the necessay libraries and load the data, here i have already converted my images and its respective labels to arrays. And we load these arrays to a label.

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np

X = image_array
y = label_array

Now we use one hot encoder to encode the labels to array of values. And we perform train test split.

In [ ]:
ct = ColumnTransformer([("OneHot", OneHotEncoder(), [0])], remainder='passthrough')
y = ct.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

The shape of our image array is flattened to fit the model as a line of arrays. We create a Neural Network with multiple layers. We use softmax activation at the last layer to get a probalistic result.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(224, 224, 3))
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Then we train the model and test it and then evaluate its accuracy.  Once done we save the model for future use.

In [ ]:

model.fit(X_train, y_train, epochs=100)
prob_arr=model.predict(X_test)
predicted_label=np.argmax(prob_arr)

print(accuracy_score(predicted_label.flatten(),y_test.flatten()))

model.save("keras_Model.h5")

After saving the model, we save our labels in a text file. Then we use opencv to help our model process real time data and make predictions.

In [ ]:
from keras.models import load_model
import cv2  # Install opencv-python
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# CAMERA can be 0 or 1 based on default camera of your computer
camera = cv2.VideoCapture(0)

while True:
    # Grab the webcamera's image.
    ret, image = camera.read()
    ret, frame = camera.read()

    # Resize the raw image into (224-height,224-width) pixels
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

    # Show the image in a window
    cv2.imshow("Webcam Image", image)

    # Make the image a numpy array and reshape it to the models input shape.
    image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

    # Normalize the image array
    image = (image / 127.5) - 1

    # Predicts the model
    prediction = model.predict(image)
    index = np.argmax(prediction)
    class_name = class_names[index].strip()
    confidence_score = prediction[0][index]

    # Print prediction and confidence score
    #print("Class:", class_name[2:], end="")
    #print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")
    cv2.putText(frame, "Class: {}".format(class_name[2:]), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, "Confidence: {}".format(str(np.round(confidence_score * 100))[:-2]), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the frame
    cv2.imshow("Classification Results", frame)

    # Listen to the keyboard for presses.
    keyboard_input = cv2.waitKey(1)

    # if q is pressed the window closes and prediction stops
    if cv2.waitKey(100) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

camera.release()
